# Meta-Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

np.random.seed(123)
tf.random.set_seed(123)

def generate_task():
    coefficients = np.random.randn(3)  
    def task(x):
        return coefficients[0] * x**2 + coefficients[1] * x + coefficients[2] + np.random.randn(*x.shape) * 0.1
    return task, coefficients

In [ ]:
def train_model_on_task(model, task, steps=50, learning_rate=0.01):
    optimizer = Adam(learning_rate)
    for _ in range(steps):
        x = np.random.uniform(-3, 3, (50, 1))
        y = task(x)
        with tf.GradientTape() as tape:
            predictions = model(x, training=True)
            loss = tf.reduce_mean((predictions - y) ** 2)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
def meta_train_model(base_model, meta_steps=50, meta_learning_rate=0.1):
    for _ in range(meta_steps):
        task, _ = generate_task()
        train_model_on_task(base_model, task, learning_rate=0.1)

In [ ]:
model = Sequential([
    Dense(10, activation='sigmoid', kernel_initializer='random_normal', input_shape=(1,)),
    Dense(1)])

model.summary()

In [ ]:
def plot_predictions(model, task, title):
    x = np.linspace(-3, 3, 100).reshape(-1, 1)
    y_true = task(x)
    y_pred = model.predict(x)
    plt.figure(figsize=(10, 5))
    plt.scatter(x, y_true, color='blue', label='True Values')
    plt.scatter(x, y_pred, color='red', label='Predictions')
    plt.title(title)
    plt.xlabel("Input Feature")
    plt.ylabel("Output Value")
    plt.legend()
    plt.show()

complex_task, _ = generate_task()
plot_predictions(model, complex_task, "Model Performance Before Meta-training on Task")

In [ ]:
meta_train_model(model)
plot_predictions(model, complex_task, "After Meta-training on Task")

# Regularization

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

def build_regularized_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape,
              kernel_regularizer=l2(0.001)), 
        Dropout(0.3),  
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),  
        Dropout(0.3),  
        Dense(16, activation='relu', kernel_regularizer=l2(0.001)), 
        Dropout(0.3), 
        Dense(1, activation='linear')  
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model
    
input_shape = (10,)
model = build_regularized_model(input_shape)
model.summary()

# Transfer Learning using Keras

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16()
base_model.summary()

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())  
model.add(layers.Dense(256, activation='relu'))  
model.add(layers.Dense(1, activation='sigmoid')) 

In [ ]:
base_model.trainable = False
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Transfer Learning using API Services

In [ ]:
import openai

openai.api_key = 'XXX' #your API key here

response = openai.File.create(
    file=open("path/to/your/dataset.csv", "rb"),
    purpose='fine-tune')
file_id = response['id']
fine_tune_response = openai.FineTune.create(training_file=file_id,
    model="gpt-4",
    n_epochs=5)

# Few-Shot Learning - Rebranding

## Product description: ZSL

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = 'XXX' ) #your API key here
base_prompt = "Write a product description for an eco-friendly kitchenware product focusing on brand ethics."

def generate_chat_completion(client, prompt, max_tokens=150, n=1, temperature=0.7, advanced_usage=False):
    if not advanced_usage:
        messages = [{"role": "user", "content": prompt}]
    else:
        messages = prompt 
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        max_tokens=max_tokens,
        n=n,
        temperature=temperature)
    return completion.choices[0].message.content

In [ ]:
zsl_response = generate_chat_completion(client, base_prompt)
print(zsl_response)

## Product description: FSL

In [ ]:
fsl_prompts = [
    {"role": "system", "content": "Describe a product using sustainable materials that supports fair labor practices."},
    {"role": "user", "content": "Our latest kitchenware range not only uses biodegradable materials but is also produced in factories ensuring fair wages and safe working conditions."},
    {"role": "system", "content": "Write about a kitchenware product that exemplifies zero waste in packaging."},
    {"role": "user", "content": "Each item in our new line of cookware comes in completely recyclable packaging, designed to leave no environmental footprint."},
    {"role": "system", "content": "Explain how a purchase supports community engagement projects."},
    {"role": "user", "content": "With every purchase, a portion of the proceeds goes towards community gardens, supporting urban greening projects."},
    {"role": "system", "content": base_prompt}]

fsl_response = generate_chat_completion(client, fsl_prompts, max_tokens=150, temperature=0.7, advanced_usage=True)
print(fsl_response)

## Email marketing campaign

In [ ]:
base_prompt = "Compose a short email to introduce our latest line of eco-friendly kitchenware. The email should be engaging, informative, and conclude with a strong call to action."

fsl_prompts = [
    {"role": "system", "content": "Make sure to highlight the eco-friendly and zero waste aspects of the kitchenware."},
    {"role": "user", "content": "Our kitchenware is designed with the planet in mind, featuring products made entirely from recycled materials and ensuring zero waste in our packaging."},
    {"role": "system", "content": "Include information about the fair labor practices in the manufacturing process."},
    {"role": "user", "content": "We are committed to ethical production practices. All our products are manufactured under fair labor conditions, ensuring all workers are paid fairly and work in safe environments."},
    {"role": "system", "content": "Mention how purchasing these products supports community engagement projects."},
    {"role": "user", "content": "Every purchase supports community initiatives, including urban greening projects and community gardens that help local environments thrive."},
    {"role": "system", "content": base_prompt}]

fsl_completion = client.chat.completions.create(
    model="gpt-4",
    messages=fsl_prompts,
    max_tokens=300,
    n=1,
    temperature=0.7
)
print(fsl_completion.choices[0].message.content)

In [ ]:
import pandas as pd

df_initial = pd.read_csv('initial_launch_data.csv')

In [ ]:
def plot_campaign_kpis(df, title='Email Campaign KPIs'):
    plt.figure(figsize=(10, 8))
    plt.plot(df['Date'], df['Open Rate (%)'], label='Open Rate (%)', marker='o')
    plt.plot(df['Date'], df['Click-Through Rate (%)'], label='Click-Through Rate (%)', marker='o')
    plt.plot(df['Date'], df['Conversion Rate (%)'], label='Conversion Rate (%)', marker='o')
    plt.plot(df['Date'], df['Unsubscribe Rate (%)'], label='Unsubscribe Rate (%)', marker='o')
    plt.xlabel('Date')
    plt.ylabel('Percentage')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_campaign_kpis(df_initial, 'Email Campaign KPIs First Week after Launch')

In [ ]:
print(f"Average Open Rate: {df_initial['Open Rate (%)'].mean():.2f}%")
print(f"Average Click-Through Rate: {df_initial['Click-Through Rate (%)'].mean():.2f}%")
print(f"Average Conversion Rate: {df_initial['Conversion Rate (%)'].mean():.2f}%")
print(f"Average Unsubscribe Rate: {df_initial['Unsubscribe Rate (%)'].mean():.2f}%")

In [ ]:
fsl_prompts = [
    {"role": "system", "content": "Compose a short email to introduce our latest line of eco-friendly kitchenware. Highlight our zero waste and ethical manufacturing processes in terms of the labor force, and emphasize the practical benefits these practices offer to the consumer. The email should be engaging, informative, and conclude with a strong call to action. Importantly, the subject line should be as exciting and engaging as possible."},
    {"role": "user", "content": "Discover our new line of eco-friendly kitchenware! Designed with sustainability at heart, each piece not only supports zero waste but is ethically made, ensuring fair labor practices and substantial benefits to the environment."},
    {"role": "system", "content": "Mention the product variety and colors."},
    {"role": "user", "content": "Now available in a range of vibrant colors, our kitchenware allows you to personalize your kitchen while maintaining a commitment to sustainability."},
    {"role": "system", "content": "Provide care instructions."},
    {"role": "user", "content": "Caring for your eco-friendly kitchenware is easy. Here are some simple tips to ensure your items last longer, enhancing their sustainability."},
    {"role": "system", "content": "Address concerns about pricing."},
    {"role": "user", "content": "Invest in quality and sustainability. While initially more expensive, our kitchenware offers long-term savings and a positive impact on the planet."}]

fsl_response = generate_chat_completion(client, fsl_prompts, max_tokens=300, temperature=0.7, advanced_usage=True)
print(fsl_response)


In [ ]:
df_improved = pd.read_csv('improved_data.csv')
plot_campaign_kpis(df_improved, 'Improved Email Campaign KPIs After FSL Refinements')

In [ ]:
print(f"Average Open Rate: {df_improved['Open Rate (%)'].mean():.2f}%")
print(f"Average Click-Through Rate: {df_improved['Click-Through Rate (%)'].mean():.2f}%")
print(f"Average Conversion Rate: {df_improved['Conversion Rate (%)'].mean():.2f}%")
print(f"Average Unsubscribe Rate: {df_improved['Unsubscribe Rate (%)'].mean():.2f}%")